In [1]:
import sys
import os
import glob
import struct
import binascii

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# import dateutil
# from scipy import __version__ as scipy_version
# import scipy.interpolate
# import scipy.stats
# from statsmodels import __version__ as statsmodels_version
# import statsmodels.formula.api as smformula

from collections import OrderedDict
from IPython.display import display, HTML

print(sys.version)
print('numpy', np.__version__)
print('matplotlib', mpl.__version__)
print('pandas',pd.__version__)
# print('scipy', scipy_version)
# print('statsmodels', statsmodels_version)

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
numpy 1.14.3
matplotlib 2.2.2
pandas 0.22.0


In [2]:
# Setup plot style
%matplotlib notebook
plt.style.use('seaborn-paper')
mpl.rcParams['figure.facecolor'] = (0.8, 0.8, 0.8, 1)

In [3]:
# Misc utility functions
def tempF2C(x): return (x-32.0)*5.0/9.0
def tempC2F(x): return (x*9.0/5.0)+32.0

In [5]:
def msbin2ieee(msbin):
    """
    Convert an array of 4 bytes containing Microsoft Binary floating point
    number to IEEE floating point format (which is used by Python)
    adapted from: https://github.com/choonkeat/ms2txt/blob/master/metastock/utils.py
    """
    as_int = struct.unpack("i", msbin)
    if not as_int:
        return 0.0
    man = int(struct.unpack('H', msbin[2:])[0])
    if not man:
        return 0.0
    exp = (man & 0xff00) - 0x0200
    man = man & 0x7f | (man << 8) & 0x8000
    man |= exp >> 1
    ieee = msbin[:2]
    ieee += bytes([man & 0xFF])
    ieee += bytes([(man >> 8) & 0xFF])
#     print(ieee)
    return struct.unpack("f", ieee)[0]

display(msbin2ieee(b'\xDB\x0F\x49\x81')*2) # should ~= pi
display(msbin2ieee(b'\xCD\xCC\x2A\x87'))

3.1415927410125732

85.4000015258789

In [7]:
FN = '../../cdfa_dd/LAAR18'

In [8]:
# 3 AS N$, 2 AS D$, 2 AS M$, 2 AS Y$, 5 AS M2$, 5 AS M3$, 16 AS X$
date_fmt = r"=hchhh" # 5 5 16"
date_size = struct.calcsize(date_fmt)

with open(FN,'rb') as fh:
    while True:
        date_bytes = fh.read(date_size)
        if len(date_bytes) < date_size:
            if len(date_bytes) > 0:
                print("WARNING: Some data left in file")
            break
        (n, _, d, m, y) = struct.unpack(date_fmt, date_bytes)
        tmin = msbin2ieee(fh.read(4))
        assert fh.read(1) == b' ' # This byte should be a space (0x20)
        tmax = msbin2ieee(fh.read(4))
        assert fh.read(1) == b' ' # This byte should be a space (0x20)
        fooX = fh.read(16) # I have no clue what this value is
        print(n, d, m, y, tmin, tmax, fooX, sep='\t')
        _ = fh.read(128-35) # skip to next record in the file (I have no idea why each record is 128 bytes)

1	1	1	18	48.5	71.9000015258789	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
2	2	1	18	52.79999923706055	72.19999694824219	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
3	3	1	18	57.099998474121094	71.5999984741211	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
4	4	1	18	54.20000076293945	75.0	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
5	5	1	18	53.900001525878906	71.80000305175781	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
6	6	1	18	57.599998474121094	68.30000305175781	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
7	7	1	18	56.70000076293945	72.5	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
8	8	1	18	60.5	64.4000015258789	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
9	9	1	18	51.20000076293945	60.5	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
10	10	1	18	47.79999923706055	66.30000305175781	b